In [1]:
import requests
import os
import re
import sys
import json
import requests
import argparse
import time
import codecs
from bs4 import BeautifulSoup
from six import u

自動點選已滿18歲

In [2]:
def fetch(url):
    response = requests.get(url)
    response = requests.get(url, cookies={'over18': '1'})  # 一直向 server 回答滿 18 歲了 !
    return response


輸出格式調整

In [44]:
widths = [
        (126,    1), (159,    0), (687,     1), (710,   0), (711,   1),
        (727,    0), (733,    1), (879,     0), (1154,  1), (1161,  0),
        (4347,   1), (4447,   2), (7467,    1), (7521,  0), (8369,  1),
        (8426,   0), (9000,   1), (9002,    2), (11021, 1), (12350, 2),
        (12351,  1), (12438,  2), (12442,   0), (19893, 2), (19967, 1),
        (55203,  2), (63743,  1), (64106,   2), (65039, 1), (65059, 0),
        (65131,  2), (65279,  1), (65376,   2), (65500, 1), (65510, 2),
        (120831, 1), (262141, 2), (1114109, 1),
]


def calc_len(string):
    def chr_width(o):
        global widths
        if o == 0xe or o == 0xf:
            return 0
        for num, wid in widths:
            if o <= num:
                return wid
        return 1
    return sum(chr_width(ord(c)) for c in string)


def pretty_print(push, title, date, author):
    pattern = '%2s\t%s%s%s\t%s'
    padding = ' ' * (50 - calc_len(title))
    print(pattern % (push, title, padding, date, author))
    
    
def pretty_print2(push, title, date, author, link):
    pattern = '%2s\t%s%s%s\t%s\t%s'
    padding = ' ' * (50 - calc_len(title))
    print(pattern % (push, title, padding, date, author, link))

In [29]:
def parse_article_meta(ent):
    ''' Step-3 (revised): parse the metadata in article entry '''
    # 基本要素都還在
    meta = {
        'title': ent.find('div.title', first=True).text,
        'push': ent.find('div.nrec', first=True).text,
        'date': ent.find('div.date', first=True).text
    }

    try:
        # 正常狀況取得資料
        meta['author'] = ent.find('div.author', first=True).text
        meta['link'] = ent.find('div.title > a', first=True).attrs['href']

    except AttributeError:
        # 但碰上文章被刪除時，就沒有辦法像原本的方法取得 作者 跟 連結
        if '(本文已被刪除)' in meta['title']:
            # e.g., "(本文已被刪除) [haudai]"
            match_author = re.search('\[(\w*)\]', meta['title'])
            match_link = re.search('\[(\w*)\]', meta['title']) 
            if match_author:
                meta['author'] = match_author.group(1)
            if match_link:
                meta['link'] = match_link.group(1)
        elif re.search('已被\w*刪除', meta['title']):
            # e.g., "(已被cappa刪除) <edisonchu> op"
            match_author = re.search('\<(\w*)\>', meta['title'])
            if match_author:
                meta['author'] = match_author.group(1)
        elif re.search('-',meta['author']):
            meta['link'] = '-'
    return meta

    # 最終仍回傳統一的 dict() 形式 paired data
    #return meta

In [45]:
start_url = 'https://www.ptt.cc/bbs/movie/index.html'
metadata = get_paged_meta(start_url, num_pages=5)
for meta in metadata:
    pretty_print(meta['push'], meta['title'], meta['date'], meta['author'])

 8	[新聞] 「終局之戰」、「亂世佳人」、「阿凡達」誰真正票房冠軍？8/06	orz44444
 6	Re: [新聞] 凱文費奇透露《雷神索爾4》為何要拍女雷神8/06	godshibainu
 7	[新聞] 《復仇者4》驚見關史黛西！「就在蜘蛛人      8/06	chufenyang
 5	Re: [新聞] 必備片單！帝國雜誌評選30年來30部經典代 8/06	Payne22
 7	Re: [新聞] 必備片單！帝國雜誌評選30年來30部經典代 8/06	Tosca
22	[公告] 板規 2019/07/05                            6/04	ckshchen
72	[片單] 有布萊德彼特吃東西的片                     8/05	isaac42
 4	[公告] lol9978 水桶7天 (板規4)                    8/05	kai3368
 6	[討論] 好的結局還算是克蘇魯嗎?                    8/05	dagayealo
 2	[好雷] 我出去透透氣，那些大人無法的事             8/05	mysmalllamb
10	Re: [新聞] 哪吒橫掃大陸票房                       8/05	suzhou
 2	Re: [新聞] 哪吒橫掃大陸票房                       8/05	rfvujm
14	[討論] 玩命關頭:特別行動片尾                      8/05	arsl400
 3	[問片] 找一部有好萊塢名字的片                     8/05	c753968412
37	[新聞] 凱文費奇透露《雷神索爾4》為何要拍女雷神    8/05	hahaha0204
47	[新聞] 必備片單！帝國雜誌評選30年來30部經典代     8/05	hahaha0204
16	[討論] 金馬義大利電影課：你最想推的幾部？         8/05	joey0602
16	[討論] 傳奇三個飛天怪獸誰強                       8/05	ultmisia
  	(已被kai3368刪除) <Qmmmmmmmm> 無關電影            8/05	Qmmmmmm

In [46]:
from requests_html import HTML

def parse_article_entries(doc):
    html = HTML(html=doc)
    post_entries = html.find('div.r-ent')
    return post_entries

In [47]:
url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
resp = fetch(url)  # step-1
post_entries = parse_article_entries(resp.text)  # step-2

for entry in post_entries:
    meta = parse_article_meta(entry)
    pretty_print2(meta['push'], meta['title'], meta['date'], meta['author'], meta['link'])  

    # pretty_print(meta['push'], meta['title'], meta['date'], meta['author'])  # for below results

 4	[問卦] 阿辰師算是台灣之光嗎                       8/06	heatthree	/bbs/Gossiping/M.1565063795.A.793.html
 2	Re: [問卦] 一句話證明自己不是1450                 8/06	erichang	/bbs/Gossiping/M.1565063865.A.474.html
 2	Re: [新聞] 為何台大電機年休學率排前四？ 過來人揭  8/06	NightStick	/bbs/Gossiping/M.1565063905.A.96C.html
  	Re: [爆卦] 柯文哲TMD創黨大會現場                  8/06	ARCHER2234	/bbs/Gossiping/M.1565063918.A.DD4.html
 1	Re: [新聞] 柯文哲問小英：民進黨去年高雄大敗 妳做  8/06	jimgene	/bbs/Gossiping/M.1565063964.A.369.html
  	Re: [新聞] 基本工資擬續漲 中小企業總會：真的快活  8/06	XDDDDDDDDDD	/bbs/Gossiping/M.1565063977.A.07A.html
  	Re: [問卦] 一句話證明自己不是1450                 8/06	opfish	/bbs/Gossiping/M.1565064030.A.808.html
  	[問卦] 辦公室電話沒接到怎麼辦?!?!                 8/06	Benegas	/bbs/Gossiping/M.1565064055.A.5CE.html
  	Fw: [其他] 大陸證監會正在統計台灣籍投資者資料     8/06	cyora	/bbs/Gossiping/M.1565064069.A.554.html
  	[公告] 八卦板板規(2019.08.01)                     8/01	arsonlolita	/bbs/Gossiping/M.1564659716.A.B48.html
 4	[公告] 版規1-3修正草案                            8/02	arsonlolita	/bbs/Gossiping/M.1

In [32]:
import re
import urllib.parse
## from utils import pretty_print

def get_metadata_from(url):

    def parse_next_link(doc):
        domain = 'https://www.ptt.cc/' 
        html = HTML(html=doc)
        controls = html.find('.action-bar a.btn.wide')
        link = controls[1].attrs.get('href')
        return urllib.parse.urljoin(domain, link)

    resp = fetch(url)
    post_entries = parse_article_entries(resp.text)
    next_link = parse_next_link(resp.text)

    metadata = [parse_article_meta(entry) for entry in post_entries]
    return metadata, next_link

def get_paged_meta(url, num_pages):
    collected_meta = []
    domain = 'https://www.ptt.cc/'
    
    for _ in range(num_pages):
        posts, link = get_metadata_from(url)
        collected_meta += posts

        url = urllib.parse.urljoin(domain, link)
    return collected_meta

#metadata = get_paged_meta(start_url, num_pages=10)
#for meta in metadata:
    #pretty_print(meta['push'], meta['title'], meta['date'], meta['author'],meta['link'])

In [48]:
def get_posts(metadata):
    # 將所有文章連結收集並串接成完整 URL
    post_links = [
        urllib.parse.urljoin(domain, meta['link'])
        for meta in metadata if 'link' in meta
    ]
    with Pool(processes=8) as pool:
        contents = pool.map(fetch, post_links)
        return contents
    

In [62]:
if __name__ == '__main__':
    pages = 5

    start = time.time()

    metadata = get_paged_meta(start_url, num_pages=pages)
    resps = get_posts(metadata)

    print('花費: %f 秒' % (time.time() - start))

    print('共%d項結果：' % len(resps))
    for post, resps in zip(metadata, resps):
        print('{0} {1: <15} {2}, 網頁內容共 {3} 字'.format(
            post['date'], post['author'], post['title'], len(resps.text)))

[<Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [404]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [404]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response [200]>, <Response

In [49]:
import time
from multiprocessing import Pool

domain = 'https://www.ptt.cc/'

In [124]:
def parse_article_content(doc,href):
    if '.html' in (href):
        html = HTML(html=doc)
        post_content = html.find('#main-content', first=True).text
        soup = BeautifulSoup(doc, 'html.parser')
        main_content = soup.find(id="main-content")
        metas = main_content.select('div.article-metaline')
        if metas:
            author = metas[0].select('span.article-meta-value')[0].string if metas[0].select('span.article-meta-value')[0] else author
            title = metas[1].select('span.article-meta-value')[0].string if metas[1].select('span.article-meta-value')[0] else title
            date = metas[2].select('span.article-meta-value')[0].string if metas[2].select('span.article-meta-value')[0] else date
            # remove meta nodes
            for meta in metas:
                meta.extract()
            for meta in main_content.select('div.article-metaline-right'):
                meta.extract()
        PTT_URL = 'https://www.ptt.cc'
        # ex. link would be <a href="/bbs/PublicServan/M.1127742013.A.240.html">Re: [問題] 職等</a>
        link = PTT_URL + href
        article_id = re.sub('\.html', '', href.split('/')[-1])        
        pushes = main_content.find_all('div', class_='push')
        for push in pushes:
            push.extract() ## 移除指定該變數
        filtered = [ v for v in main_content.stripped_strings if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--'] ]
        expr = re.compile(u(r'[^\u4e00-\u9fa5\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\s\w:/-_.?~%()]'))
        for i in range(len(filtered)):
            filtered[i] = re.sub(expr, '', filtered[i])

        filtered = [_f for _f in filtered if _f]  # remove empty strings
        filtered = [x for x in filtered if article_id not in x]  # remove last line containing the url of the article
        content = ' '.join(filtered)
        content = re.sub(r'(\s)+', ' ', content)
        meta = {
            'content':content,
            'article_id':article_id,
            'author': author,
            'title' : title,
            'date' : date
        }
        return meta
    else:
        meta = {
            'content':'被刪除',
            'article_id': '無',
            'link':'無'
        }
        return meta
    

In [125]:
if __name__ == '__main__':
    pages = 2
    start_url = 'https://www.ptt.cc/bbs/movie/index.html'
    start = time.time()

    metadata = get_paged_meta(start_url, num_pages=pages)
    resps = get_posts(metadata)

    #contents = [parse_article_content(content.text, metadata['link']) for content, metadata in zip(resps, metadata)]
    contents = []
    for content, metadata in zip(resps, metadata):
        try:
            contents.append(parse_article_content(content.text, metadata['link']))
        except KeyError:
            contents.append([content.text,'test'])
    
    
    print('花費: %f 秒' % (time.time() - start))

    print('共%d項結果：' % len(resps))
    ##for post, resps, text in zip(metadata, resps,contents):
    ##    print('{0} {1: <15} {2}, 網頁內容共 {3} 字, {4}'.format(
    ##        post['date'], post['author'], post['title'], len(resps.text), text['content']))

花費: 4.387905 秒
共35項結果：


In [126]:
contents[9]

{'content': '漫威《永恆族》有華星傳她和裘莉宇宙對尬 https://i.imgur.com/pHmFN69.jpg 繼梁朝偉將演出漫威電影《上氣》的滿大人之後，又一位華裔明星可望加入漫 威宇宙系列，美國綜藝報（Variety）快報：華裔女星陳靜將演出《永恆族》，和 裘莉大飆戲。 據報導，陳靜曾在漫威《驚奇隊長》中飾女反派密涅瓦，不過她在《永恆族》的 角色卻可能是另一位，如果成真，她將成為漫威電影第1位在不同電影演不同角色的 演員。但至截稿前尚未獲得漫威證實。 英文名Gemma Chan的陳靜，父親是香港人，母親是移民蘇格蘭的中國人，百分百華裔血統 的陳靜生於英國，畢業於牛津大學伍斯特學院法律系，後來更獲得某大律師事務所聘用， 但熱愛演戲勝過當律師的陳靜，仍義無反顧踏入演藝圈。 今年36歲的陳靜2006年出道，努力多年，2018年才在美國大賣的《瘋狂亞洲富豪》中飾演 老公出軌的闊太，受到好萊塢矚目；在《驚奇隊長》的戲分不多但很搶眼；如果確定能演 《永恆族》和裘莉同台飆戲，肯定是這部明年11月6日在美公映超級大片的一大亮點。 https://reurl.cc/pO4yl 蠻喜歡密涅瓦這個角色，漫畫裡也曾是驚奇隊長宿敵之一，本來期望她沒有死透，繼續擔 綱演出反派，但要在《永恆族》演出同一個角色可能性好像較小。 ',
 'article_id': 'M.1565066458.A.A62',
 'author': 'CYKONGG (CYKONGG)',
 'title': '[新聞] 《永恆族》有華星！傳她再加入漫威宇宙',
 'date': 'Tue Aug  6 12:40:51 2019'}

In [127]:
contents[3]

{'content': '年度片單有黑豹很正常吧 你看那一年聲量最大 又得獎的片有誰 非黑豹莫屬阿 其他根本沒什麼印象 覺得不適合的的只是不喜歡黑豹而已吧 不然鄉民舉幾個聲量比它大 討論度比它多的 另外 沒有Save private ryan 跟the forrest gump 我比較意外 ',
 'article_id': 'M.1565031671.A.280',
 'author': 'Payne22 (培恩22)',
 'title': 'Re: [新聞] 必備片單！帝國雜誌評選30年來30部經典代',
 'date': 'Tue Aug  6 03:01:08 2019'}

In [130]:
z= zip(metadata , contents)

In [136]:
contents[0]['article_id']

'M.1565026014.A.B3C'

In [134]:
filename = './crawl_file/'+ contents['article_id']+'.json'
with open(filename, 'w') as outfile:
    json.dump(test, outfile, ensure_ascii=False)

TypeError: list indices must be integers or slices, not str

### 匯入MongoDB

In [1]:
import pymongo
from pymongo import MongoClient


In [6]:
client = pymongo.MongoClient(host='localhost', port=27017)

In [7]:
db = client.ptt

### show database

In [8]:
if __name__ == '__main__':
    client = pymongo.MongoClient("localhost", 27017, maxPoolSize=50)
    d = dict((db, [collection for collection in client[db].collection_names()])
             for db in client.database_names())
    print(json.dumps(d))

/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  after removing the cwd from sys.path.
/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  after removing the cwd from sys.path.


NameError: name 'json' is not defined

### show db lists of mongodb

In [9]:
MongoClient().database_names()

/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  """Entry point for launching an IPython kernel.


['admin', 'config', 'local', 'ptt', 'test']

In [2]:
pymongo.version

'3.8.0'

In [13]:
db.list_collection_names()

['article']

In [12]:
#db.article.drop()
collection = db['article']

In [11]:
result = collection.find({}, {'title':1})
print(type(result))
print(result[0])

NameError: name 'collection' is not defined

In [23]:
for x, y in zip(metadata, contents):
    z = {**x, **y} 
    result = collection.insert([z])    
    #filename = './crawl_file/'+ z['article_id']+'.json'
    #with open(filename, 'w') as outfile:
        #json.dump(z, outfile, ensure_ascii=False)

/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
result = collection.find({}, {'title':1, '_id':0})
print(type(result))


<class 'pymongo.cursor.Cursor'>


In [25]:
for i in result:
    print(i)

{'title': 'Re: [討論] 劉德華主演警匪片經典《五億探長雷洛傳》上映27週年'}
{'title': '[討論] 布麗拉森哪一部影片的扮裝最讓你驚豔'}
{'title': '[好雷] 我很好，讓死亡證明自己活著'}
{'title': '[討論] 威秀寶寶是不是接到置入了?'}
{'title': '(本文已被刪除) [tontontonni]'}
{'title': 'Re: [討論] 劉德華主演警匪片經典《五億探長雷洛傳》上映27週年'}
{'title': '[好雷] 《我很好》 無彩青春的輓歌'}
{'title': '[版規] 電影版版規 201808'}
{'title': '[公告] 電影版板規修訂說明'}
{'title': '[公告] 板規修訂 板規參議規範'}
{'title': '[超級好雷] 藍色項圈'}
{'title': '[好雷] 《裸愛殺機》'}
{'title': '[ 普雷] 鬼修女'}
{'title': '[新聞] 賈樟柯《在清朝》2019年開機'}
{'title': '[討論] 與神同行快下檔了'}
{'title': '[討論] 老人家看終極戰士(無雷)'}
{'title': '[新聞] 粽邪 票房近5000萬 海外版權9國爭搶'}
{'title': '[片單] 校園青少女魯蛇'}
{'title': '[討論] 投名狀和讓子彈飛哪一部更為經典？'}
{'title': '[請益] 已看舊版惡魔島值得花錢再最新版嗎??'}
{'title': '[新聞] 黑木瞳昔日母女變情敵 《退》戰小三廣末'}
{'title': '[問片] 一部有關批踢踢的微電影 (有雷 (已解'}
{'title': '[問片] 請問跟「攝影」相關的電影'}
{'title': '[新聞] 狂穿褲裝被取笑 布蕾克萊佛莉「神反擊」'}
{'title': '[情報] 《猛禽小隊》演員候選人名單曝光'}
{'title': '[好雷] 《一屍到底》，電影的重新定義。'}
{'title': '[選片] 惡魔島VS終極戰士'}
{'title': '[討論] 睡著也好醒來也罷'}
{'title': '[片單] 看完影評後整個劇情反轉的電影'}
{'title': '[好雷] 血薄荷—硬派媽媽的復仇之路'}
{'title': '[

In [52]:
client.drop_database('article')
